In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"  # specify which GPU(s) to be used

import sys
sys.argv = ['GPT_eval_multi.py']

import clip
import torch
import numpy as np
import models.vqvae as vqvae
import models.t2m_trans as trans
import warnings
warnings.filterwarnings('ignore')


# from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, T5ForConditionalGeneration, T5Tokenizer, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer


In [16]:
# tokenizer = T5Tokenizer.from_pretrained("checkpoints/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("checkpoints/flan-t5-base", legacy=True)
model = T5ForConditionalGeneration.from_pretrained("checkpoints/flan-t5-base").to('cuda')

# input_text = "translate English to German: How old are you?"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))

In [17]:
import codecs as cs

split_file = '/home/tuan/tdngo/motion_ws/T2M-GPT/dataset/HumanML3D/train_val.txt'

id_list = []
with cs.open(split_file, 'r') as f:
    for line in f.readlines():
        id_list.append(line.strip())

In [18]:
from tqdm import tqdm

new_name_list = []
data_dict = {}
for name in tqdm(id_list):
    # Read text
    try:
        with cs.open(os.path.join('/home/tuan/tdngo/motion_ws/T2M-GPT/dataset/HumanML3D/texts', name + '.txt')) as f:
            text_data = []
            flag = False
            lines = f.readlines()

            for line in lines:
                try:
                    text_dict = {}
                    line_split = line.strip().split('#')
                    caption = line_split[0]
                    t_tokens = line_split[1].split(' ')
                    f_tag = float(line_split[2])
                    to_tag = float(line_split[3])
                    f_tag = 0.0 if np.isnan(f_tag) else f_tag
                    to_tag = 0.0 if np.isnan(to_tag) else to_tag

                    text_dict['caption'] = caption
                    text_dict['tokens'] = t_tokens
                    if f_tag == 0.0 and to_tag == 0.0:
                        flag = True
                        text_data.append(text_dict)
                except:
                    pass
        if flag:
            data_dict[name] = text_data
            new_name_list.append(name)
    except:
        pass

    
                        

100%|██████████| 24844/24844 [00:14<00:00, 1764.36it/s]


In [22]:
for name, text_data in tqdm(data_dict.items()):
    # print(text_data)
    emb_list = []
    for t in text_data:
        text = t['caption']
        # print(text)

        input_ids = tokenizer([text], return_tensors="pt",)
        # print(input_ids)
        # forward pass through encoder only
        output = model.encoder(
            input_ids=input_ids["input_ids"].cuda(), 
            attention_mask=input_ids["attention_mask"].cuda(), 
            return_dict=True
        )
        # get the final hidden states
        emb = output.last_hidden_state
        emb_list.append(emb)
    torch.save(emb_list, os.path.join('/home/tuan/tdngo/motion_ws/T2M-GPT/dataset/HumanML3D/flan-t5-base_text_embeddings', name + '.pth'))


  0%|          | 0/24844 [00:00<?, ?it/s]

 28%|██▊       | 6839/24844 [07:14<21:51, 13.73it/s]

In [15]:
input_text = ["translate English to German: How old are you?", "translate English?"]

input_ids = tokenizer(input_text, return_tensors="pt", max_length=11, padding=True)
print(input_ids)
# forward pass through encoder only
output = model.encoder(
    input_ids=input_ids["input_ids"].cuda(), 
    attention_mask=input_ids["attention_mask"].cuda(), 
    return_dict=True
)
# get the final hidden states
emb = output.last_hidden_state
print(emb.shape)

{'input_ids': tensor([[13959,  1566,    12,  2968,    10,   571,   625,    33,    25,    58,
             1],
        [13959,  1566,    58,     1,     0,     0,     0,     0,     0,     0,
             0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}
torch.Size([2, 11, 768])
